In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Locate the data
data = '../Nasdaq Std.csv'
# Read data with pandas
dataset = pd.read_csv(data)
# Extract the values of the desired column
full_set = dataset['NASDAQ_100'].values
dataset

,Unnamed: 0,Date,NASDAQ_100,NAS_Returns_Rolling_Std
0,0,1979-12-28,0.399392,NaN
1,1,1979-12-31,0.205527,NaN
2,2,1980-01-02,-1.965066,NaN
3,3,1980-01-03,-1.484779,NaN
4,4,1980-01-04,1.404400,NaN
...,...,...,...,...
10366,10366,2021-02-08,0.947871,1.014844
10367,10367,2021-02-09,0.143416,1.014787
10368,10368,2021-02-10,-0.251076,1.013661
10369,10369,2021-02-11,0.381028,1.012610


In [4]:
# We split our data on the 7200 mark as it represents 70% of it
train_data = full_set[5061:9061]
test_data = full_set[9061:]
# dataset.iloc[5061]

In [5]:
# Data Normalization
from sklearn.preprocessing import MinMaxScaler
# Data must be reshaped in order to use fit_transform
training_set = train_data.reshape(-1,1)
test_set = test_data.reshape(-1,1)

sc = MinMaxScaler(feature_range=(0,1))

X_train_scaled = sc.fit_transform(training_set)

In [6]:
# Create lists for x and y train
X_train = []

y_train = []

# We will run our model on 100 timesteps
for i in range(20,4000):
    
    X_train.append(X_train_scaled[i- 20:i,0])

    y_train.append(X_train_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [7]:
# Import the necessary models from keras within tensor flow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [8]:
# Create a callback function with keras, which will monitor thr rmse of each epoch 
# And stop our model on the epoch that offers the best balance so we don't overfit our model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [9]:
# Create the layers of our model droping out and replenishing 20% of our data after each layer
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
# Include a printout of the model summary 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [10]:
model.compile(optimizer='adam',loss= 'mean_squared_error',
             metrics=[tf.keras.metrics.RootMeanSquaredError()])
# While we git our model we assign it to a variable 
history = model.fit(X_train,y_train,epochs=20,batch_size=100,callbacks=[callback], verbose = 2)

Epoch 1/20
40/40 - 12s - loss: 0.0238 - root_mean_squared_error: 0.1544
Epoch 2/20
40/40 - 2s - loss: 0.0065 - root_mean_squared_error: 0.0807
Epoch 3/20
40/40 - 2s - loss: 0.0063 - root_mean_squared_error: 0.0796
Epoch 4/20
40/40 - 2s - loss: 0.0062 - root_mean_squared_error: 0.0787
Epoch 5/20
40/40 - 2s - loss: 0.0062 - root_mean_squared_error: 0.0788
Epoch 6/20
40/40 - 2s - loss: 0.0059 - root_mean_squared_error: 0.0770
Epoch 7/20
40/40 - 2s - loss: 0.0059 - root_mean_squared_error: 0.0769
Epoch 8/20
40/40 - 2s - loss: 0.0059 - root_mean_squared_error: 0.0767
Epoch 9/20
40/40 - 2s - loss: 0.0059 - root_mean_squared_error: 0.0769
Epoch 10/20
40/40 - 2s - loss: 0.0059 - root_mean_squared_error: 0.0767
Epoch 11/20
40/40 - 2s - loss: 0.0058 - root_mean_squared_error: 0.0764
Epoch 12/20
40/40 - 2s - loss: 0.0058 - root_mean_squared_error: 0.0759
Epoch 13/20
40/40 - 3s - loss: 0.0057 - root_mean_squared_error: 0.0757
Epoch 14/20
40/40 - 2s - loss: 0.0058 - root_mean_squared_error: 0.0760


In [11]:
# Print out the number of epochs used for this model
print(len(history.history['loss']))

20


In [12]:
# Making Predictions to the test set
dataset_total = np.concatenate((train_data, test_data), axis = 0)
# Prepare our data to the test
inputs = dataset_total[len(dataset_total) - len(test_data) - 100:]
# Reshape our data to fit the upcoming for loop
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

# Create lists to which we'll append x and y values for future test
X_test = []
y_test = []

# Create a loop that will run through groups of 200 timesteps for the test sample
for i in range(20, 1410):
    X_test.append(inputs[i-20:i,0])
    y_test.append(inputs[i, 0])

In [13]:
# Turn our lists into a numpy numbers array
X_test = np.array(X_test)
y_test = np.array(y_test)

# Reshape our array into 3 dimensions so it can fit the LSTM Model
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_test = y_test.reshape(-1, 1)

In [14]:
 model_loss, rmse = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Root Mean Square Error:{rmse}")

44/44 - 3s - loss: 0.0032 - root_mean_squared_error: 0.0569
Normal Neural Network - Loss: 0.003239464247599244, Root Mean Square Error:0.05691629275679588


In [36]:
predicted_return = model.predict(X_test)
predicted_return = sc.inverse_transform(predicted_return)

In [37]:
predicted_return

array([[-0.10127297],
       [-0.1112603 ],
       [-0.11983223],
       ...,
       [-0.07951628],
       [-0.07655476],
       [-0.0742242 ]], dtype=float32)

In [38]:
y_test = sc.inverse_transform(y_test)


In [41]:
predicted_return = predicted_return *100

In [39]:

import plotly.express as px

In [42]:
output = pd.DataFrame({
    'predicted_returns' : [x[0] for x in predicted_return],
    'actual_returns' : [x[0] for x in y_test]
})
output

,predicted_returns,actual_returns
0,-10.127297,-385.198573
1,-11.126030,416.428462
2,-11.983223,-964.501635
3,-12.443226,-154.411485
4,-12.575812,-362.221168
...,...,...
1385,-8.815785,298.613171
1386,-8.893802,-158.660737
1387,-7.951628,-382.900630
1388,-7.655476,-23.595740


In [33]:
px.line(output, y=["predicted_returns", "actual_returns"])

In [35]:
output.to_csv('../../Model Results/Nasdaq-Returns-Predicted.csv')
